## Métodos Numéricos II: Ejercicios Práctica 3

#### Autor: Juan Manuel Rodríguez Gómez

In [1]:
import sympy as sp

import numpy as np

from tabulate import tabulate # Si no se encuentra el módulo, hacer "conda install tabulate" en la consola con el actual
                              # environment activado (se puede abrir desde Anaconda)

## Métodos de Euler

#### 1. Repita las aproximaciones anteriores con diferentes valores de $N$ (y por tanto de $h$) y compruebe el efecto en cuanto a mayor o menor precisión, estabilidad y coste computacional.

In [2]:
t, z = sp.symbols('t, z')

def f(t,z):
    return z # Ecuacion y'(t)=y(t) ( la solución exacta es exp(t) )

In [3]:
def euler_explicito(F,x0,y0,xfinal,N):
    ''' método de Euler explicito para resolver el PVI
    X,Y     = euler_explicito(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    for n in range(N):
        Y.append(Y[n] + h*F(X[n],Y[n]))
        
    return np.array(X),np.array(Y)

In [4]:
a = 0; b = 1   # Extremos inferior y superior del intervalo 
ya = 1          # Condición inicial del PVI

In [5]:
%%timeit # Veamos cuánto tarda en ejecutarse para N = 10
N = 10
xx, yEulerexpl= euler_explicito(f,a,ya, b, N)

30.4 µs ± 500 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
%%timeit # Veamos cuánto tarda en ejecutarse para N = 100
N = 100
xx, yEulerexpl= euler_explicito(f,a,ya, b, N)

64.1 µs ± 6.19 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
%%timeit # Veamos cuánto tarda en ejecutarse para N = 1000
N = 1000
xx, yEulerexpl= euler_explicito(f,a,ya, b, N)

341 µs ± 4.16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Como era de esperar, a mayor N, mayor tiempo de cómputo (esto ocurre así también con el resto de métodos)

In [8]:
def euler_implicito(F,x0,y0,xfinal,N):
    ''' método de Euler implicito para resolver el PVI
    X,Y     = euler_implicito(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    y1 = sp.Symbol('y1') # Usaremos esta variable como incógnita de la ecuación a resolver en cada iteración
    
    for n in range(N):   
        Y.append(sp.solve(Y[n]+h*F(X[n+1],y1)-y1,y1)[0]) # y1 es yn+1, lo demas es resultado de despejar
        
    return np.array(X),np.array(Y)

In [9]:
%%timeit # Veamos cuánto tarda en ejecutarse para N = 100
N = 100
xx, yEulerimpl= euler_implicito(f,a,ya, b, N)

2.13 s ± 64.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Nótese el aumento del tiempo de cómputo comparado con Euler Explícito para N = 100. Esto es así porque en Euler Implícito tenemos que resolver una ecuación en cada iteración.

In [10]:
def euler_implicito_aprox(F,x0,y0,xfinal,N):    
    ''' método de Euler implicito aproximado para resolver el PVI
    X,Y     = euler_implicito_aprox(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    for n in range(N):   
        y1 = Y[n] + h*F(X[n],Y[n])    # Valor inicial para una iteración de punto fijo
        Y.append(Y[n]+h*F(X[n+1],y1)) 
        
    return np.array(X),np.array(Y)

In [11]:
%%timeit # Veamos cuánto tarda en ejecutarse para N = 100
N = 100
xx, yEulerimplaprox= euler_implicito_aprox(f,a,ya, b, N)

84.2 µs ± 751 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Nótese la disminución del tiempo de cómputo comparado con Euler Implícito para N = 100.

In [12]:
ENES = [10,50,100]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yEulerexpl = euler_explicito(f,a,ya, b, N)
    xx, yEulerimpl = euler_implicito(f,a,ya, b, N)
    xx, yEulerimplaprox = euler_implicito_aprox(f,a,ya, b, N)
  
    error_expl = abs(yEulerexpl[N]-valor_real)
    error_impl = abs(yEulerimpl[N]-valor_real)
    error_impl_aprox = abs(yEulerimplaprox[N]-valor_real)
 
    
    errores.append([N, error_expl, error_impl, error_impl_aprox])
    

print(tabulate(errores, headers=["N","Error Euler expl", "Error Euler impl", "Error Euler impl aprox"]))

  N    Error Euler expl    Error Euler impl    Error Euler impl aprox
---  ------------------  ------------------  ------------------------
 10           0.124539            0.14969                   0.121139
 50           0.0266938           0.0276909                 0.0265927
100           0.013468            0.0137172                 0.013444


Aunque a mayor N aumente el tiempo de cómputo (debido a la realización de más operaciones), como h es menor, entonces obtenemos valores más aproximados (el error es menor), es decir, tenemos más precisión.

Obsérvese también que los tres métodos nos dan un valor de error parecido.

Veamos ahora la estabilidad. Para ello, ponemos una nueva condición inicial muy cercana a la que teníamos y ejecutamos los métodos.

In [13]:
ya_perturbado = 0.999

In [14]:
ENES = [10,50,100]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yEulerexpl = euler_explicito(f,a,ya, b, N)
    xx, yEulerexpl_perturbado = euler_explicito(f,a,ya_perturbado, b, N)

    xx, yEulerimpl = euler_implicito(f,a,ya, b, N)
    xx, yEulerimpl_perturbado = euler_implicito(f,a,ya_perturbado, b, N)

    xx, yEulerimplaprox = euler_implicito_aprox(f,a,ya, b, N)
    xx, yEulerimplaprox_perturbado = euler_implicito_aprox(f,a,ya_perturbado, b, N)

    error_expl = abs(yEulerexpl[N]-valor_real)
    error_expl_perturbado = abs(yEulerexpl_perturbado[N]-valor_real)

    error_impl = abs(yEulerimpl[N]-valor_real)
    error_impl_perturbado = abs(yEulerimpl_perturbado[N]-valor_real)

    error_impl_aprox = abs(yEulerimplaprox[N]-valor_real)
    error_impl_aprox_perturbado = abs(yEulerimplaprox_perturbado[N]-valor_real)

    
    errores.append([N, error_expl, error_expl_perturbado, error_impl, error_impl_perturbado, error_impl_aprox, error_impl_aprox_perturbado])
    

print(tabulate(errores, headers=["N","EulerExpl", "EulerExplPerturbado", "EulerImpl", "EulerImplPerturbado", "EulerImplAprox", "EulerImplAproxPerturbado"]))

  N    EulerExpl    EulerExplPerturbado    EulerImpl    EulerImplPerturbado    EulerImplAprox    EulerImplAproxPerturbado
---  -----------  ---------------------  -----------  ---------------------  ----------------  --------------------------
 10    0.124539               0.127133     0.14969                0.146822          0.121139                    0.1183
 50    0.0266938              0.0293854    0.0276909              0.0249449         0.0265927                   0.0238478
100    0.013468               0.0161728    0.0137172              0.0109852         0.013444                    0.0107123


Con la perturbación introducida, comprobamos que el error no aumenta mucho con respecto a los resultados sin perturbación. Por tanto, todos estos métodos de Euler son estables.

#### 2. A partir de la implementación del algoritmo del método de Euler explícito, realice las modificaciones oportunas para obtener también las implementaciones correspondientes a los métodos de Euler mejorado (o del punto medio), así como del de Euler modificado (o de Heun).

In [15]:
def euler_mejorado(F,x0,y0,xfinal,N):    
    ''' método de Euler mejorado (o del punto medio) aproximado para resolver el PVI
    X,Y     = euler_mejorado(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    for n in range(N):
        Y.append(Y[n] + h*F(X[n]+(h/2),Y[n]+(h/2)*F(X[n],Y[n])))
        
    return np.array(X),np.array(Y)

In [16]:
def euler_modificado(F,x0,y0,xfinal,N):    
    ''' método de Euler modificado (o de Heun) aproximado para resolver el PVI
    X,Y     = euler_modificado(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    for n in range(N):
        Y.append(Y[n] + (h/2)*(F(X[n],Y[n])+F(X[n],Y[n]+h*F(X[n],Y[n]))))
        
    return np.array(X),np.array(Y)

In [17]:
ENES = [10,50,100]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yEulerexpl = euler_explicito(f,a,ya, b, N)
    xx, yEulerimpl = euler_implicito(f,a,ya, b, N)
    xx, yEulerimplaprox = euler_implicito_aprox(f,a,ya, b, N)
    xx, yEulermej = euler_mejorado(f,a,ya, b, N)
    xx, yEulermod = euler_modificado(f,a,ya, b, N)
    
    error_expl = abs(yEulerexpl[N]-valor_real)
    error_impl = abs(yEulerimpl[N]-valor_real)
    error_impl_aprox = abs(yEulerimplaprox[N]-valor_real)
    error_mej = abs(yEulermej[N]-valor_real)
    error_mod = abs(yEulermod[N]-valor_real)
    
    errores.append([N, error_expl, error_impl, error_impl_aprox, error_mej, error_mod])
    

print(tabulate(errores, headers=["N","Error Euler expl", "Error Euler impl", "Error Euler impl aprox", "Error Euler mejorado", "Error Euler modificado"]))

  N    Error Euler expl    Error Euler impl    Error Euler impl aprox    Error Euler mejorado    Error Euler modificado
---  ------------------  ------------------  ------------------------  ----------------------  ------------------------
 10           0.124539            0.14969                   0.121139              0.00420098                0.00420098
 50           0.0266938           0.0276909                 0.0265927             0.000178516               0.000178516
100           0.013468            0.0137172                 0.013444              4.49659e-05               4.49659e-05


Obsérvese como los Métodos de Euler Mejorado y de Euler Modificados nos dan un valor de error mucho menor comparado con el resto de Métodos de Euler.

## Métodos de Taylor

#### 1. Realice varios experimentos numéricos, con diferentes valores de $N$, y compare y ratifique los órdenes de convergencia de los diferentes métodos vistos hasta el momento.

In [18]:
def taylor2(F,x0,y0,xfinal,N): 
    ''' método de Taylor de orden p = 2 para resolver el PVI
    X,Y     = taylor2(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    t, z = sp.symbols('t, z')
    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    def F1(t,z):
        return F(t,z)

    def F2(t,z):
        return sp.diff(F(t,z),t) + sp.diff(F(t,z),z)*F1(t,z)
    
    for n in range(N):
        Y.append(Y[n]+h*F1(X[n],Y[n])+h**2/2*F2(t,z).subs({t:X[n],z:Y[n]}))

    return np.array(X),np.array(Y)

In [19]:
ENES = [10,50,100]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yTaylor2 = taylor2(f,a,ya,b,N)
    
    error_taylor = abs(yTaylor2[N]-valor_real)
    
    errores.append([N, error_taylor])
    

print(tabulate(errores, headers=["N", "Error Taylor"]))

  N    Error Taylor
---  --------------
 10     0.00420098
 50     0.000178516
100     4.49659e-05


A mayor N, como h es menor, obtenemos valores más aproximados (el error es menor), es decir, tenemos más precisión.

Vemos que obtenemos los mismos valores de error que obtuvimos con Euler Mejorado y Euler Modificado.

#### 2. A partir de la implementación del algoritmo del método de Taylor de orden $p=2$, realice las modificaciones oportunas para obtener también una posible implementación correspondiente al método de Taylor de orden $p=3$.

In [20]:
def taylor3(F,x0,y0,xfinal,N): 
    ''' método de Taylor de orden p = 3 para resolver el PVI
    X,Y     = euler_modificado(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    t, z = sp.symbols('t, z')
    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    def F1(t,z):
        return F(t,z)

    def F2(t,z):
        return sp.diff(F(t,z),t) + sp.diff(F(t,z),z)*F1(t,z)
    
    def F3(t,z):
        return sp.diff(F2(t,z),t) + sp.diff(F2(t,z),z)*F1(t,z)
    
    for n in range(N):
        Y.append(Y[n]+h*F1(X[n],Y[n])+ h**2/2*F2(t,z).subs({t:X[n],z:Y[n]}) + h**3/6*F3(t,z).subs({t:X[n],z:Y[n]}))
        
    return np.array(X),np.array(Y)

In [21]:
ENES = [10,50,100,500]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yTaylor2 = taylor2(f,a,ya,b,N)
    xx, yTaylor3 = taylor3(f,a,ya,b,N)
    
    error_taylor2 = abs(yTaylor2[N]-valor_real)
    error_taylor3 = abs(yTaylor3[N]-valor_real)
    
    errores.append([N, error_taylor2, error_taylor3])
    

print(tabulate(errores, headers=["N", "Error Taylor2", "Error Taylor3"]))

  N    Error Taylor2    Error Taylor3
---  ---------------  ---------------
 10      0.00420098       0.000104566
 50      0.000178516      8.91716e-07
100      4.49659e-05      1.12359e-07
500      1.80947e-06      9.04639e-10


Obsérvese que el método de Taylor de orden p = 3 es mucho más preciso que el de orden p = 2 (para N = 50 ya obtenemos una solución muy aproximada)

## Métodos de Runge-Kutta

#### 1. Repita las aproximaciones anteriores con diferentes valores de $N$ (y por tanto de $h$) y compruebe el efecto en cuanto a mayor o menor precisión, estabilidad y coste computacional.

In [22]:
def RK4(F,x0,y0,xfinal,N):
    ''' método de Runge-Kutta de 4 evaluaciones para resolver el PVI
    X,Y     = RK4(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    def ecuacionesRK4(F,xn,yn,h):
        K1 = F(xn,yn)
        K2 = F(xn + h/2, yn + K1*h/2)
        K3 = F(xn + h/2, yn + K2*h/2)
        K4 = F(xn + h  , yn + K3*h)
        return (K1 + 2*K2 + 2*K3 + K4)/6    
    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    for n in range(N):
        Y.append( Y[n] + h*ecuacionesRK4(F,X[n],Y[n],h) )
        
    return np.array(X),np.array(Y)

In [23]:
ENES = [10,50,100,500]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yEulerexpl = euler_explicito(f,a,ya, b, N)
    xx, yEulerimpl = euler_implicito(f,a,ya, b, N)
    xx, yEulerimplaprox = euler_implicito_aprox(f,a,ya, b, N)
    xx, yEulermej = euler_mejorado(f,a,ya, b, N)
    xx, yEulermod = euler_modificado(f,a,ya, b, N)
    xx, yTaylor2 = taylor2(f,a,ya,b,N)
    xx, yTaylor3 = taylor3(f,a,ya,b,N)
    xx, yRK4 = RK4(f,a,ya,b,N)
    
    error_expl = abs(yEulerexpl[N]-valor_real)
    error_impl = abs(yEulerimpl[N]-valor_real)
    error_impl_aprox = abs(yEulerimplaprox[N]-valor_real)
    error_mej = abs(yEulermej[N]-valor_real)
    error_mod = abs(yEulermod[N]-valor_real)
    error_taylor2 = abs(yTaylor2[N]-valor_real)
    error_taylor3 = abs(yTaylor3[N]-valor_real)
    error_RK4 = abs(yRK4[N]-valor_real)
    
    errores.append([N, error_expl, error_impl, error_impl_aprox, error_mej, error_mod, error_taylor2, error_taylor3, error_RK4])
    

print(tabulate(errores, headers=["N","Euler expl", "Euler impl", "Euler impl aprx", "Euler mejorado", "Euler modificado", "Taylor2", "Taylor3", "RK4"]))

  N    Euler expl    Euler impl    Euler impl aprx    Euler mejorado    Euler modificado      Taylor2      Taylor3          RK4
---  ------------  ------------  -----------------  ----------------  ------------------  -----------  -----------  -----------
 10    0.124539      0.14969            0.121139         0.00420098          0.00420098   0.00420098   0.000104566  2.08432e-06
 50    0.0266938     0.0276909          0.0265927        0.000178516         0.000178516  0.000178516  8.91716e-07  3.56449e-09
100    0.013468      0.0137172          0.013444         4.49659e-05         4.49659e-05  4.49659e-05  1.12359e-07  2.24641e-10
500    0.00271331    0.00272327         0.00271239       1.80947e-06         1.80947e-06  1.80947e-06  9.04639e-10  3.61045e-13


A mayor N, como h es menor, obtenemos valores más aproximados (el error es menor), es decir, tenemos más precisión.

Comparando el método de Runge-Kutta de 4 evaluaciones con el resto de métodos, podemos observar que con este método es el que se comete un error más pequeño

Veamos ahora la estabilidad. Para ello, ponemos una nueva condición inicial muy cercana a la que teníamos y ejecutamos el método.

In [24]:
ya_perturbado = 0.999

In [25]:
ENES = [10,50,100]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yRK4 = RK4(f,a,ya,b,N)
    xx, yRK4_perturbado = RK4(f,a,ya_perturbado,b,N)
    
    error_RK4 = abs(yRK4[N]-valor_real)
    error_RK4_perturbado = abs(yRK4_perturbado[N]-valor_real)

    errores.append([N, error_RK4, error_RK4_perturbado])
    

print(tabulate(errores, headers=["N","RK4", "RK4 Perturbado"]))

  N          RK4    RK4 Perturbado
---  -----------  ----------------
 10  2.08432e-06        0.00272036
 50  3.56449e-09        0.00271829
100  2.24641e-10        0.00271828


Con la perturbación introducida, comprobamos que el error aumenta un poco con respecto a los resultados sin perturbación pero tampoco se dispara. Por tanto, el método de Runge-Kutta de 4 evaluaciones es estable.

#### 2. A partir de la implementación del algoritmo del método de Runge-Kutta de 4 evaluaciones, realice las modificaciones oportunas para intentar obtener también las implementaciones correspondientes a otros muchos métodos de tipo Runge-Kutta, tanto explícitos como implícitos, a partir del correspondiente arreglo de Butcher.

In [26]:
def RK2(F,x0,y0,xfinal,N,butcher):
    ''' método de Runge-Kutta de 2 evaluaciones para resolver el PVI
    X,Y     = RK2(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    butcher = Arreglo de Butcher (determina cualquier método de Runge-Kutta)
    '''
    def ecuacionesRK(xn,yn):
        K=[]
        for i in range(butcher[0]):
            aux = 0
            for j in range(i):
                aux = aux + butcher[1][i][j]*K[j]
            K.append(F(xn + butcher[3][i]*h,yn + h*aux))
            
        suma = 0
        for i in range(butcher[0]):
            suma += butcher[2][i]*K[i]
        return suma
    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    for n in range(N):
        Y.append(Y[n] + h * ecuacionesRK(X[n],Y[n]))
        
    return np.array(X),np.array(Y)

In [27]:
butcher = [2, [[0,0],[1,0]],[1/2,1/2],[0,1]] # Primero le indicamos la dimensión de la matriz, luego las a_ij, luego las b_i
                                             # y luego las c_i, con i,j = 1...m (siendo m el número de evaluaciones del método)
ENES = [10,50,100,500]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yRK4 = RK4(f,a,ya,b,N)
    xx, yRK2 = RK2(f,a,ya,b,N,butcher)
    
    error_RK4 = abs(yRK4[N]-valor_real)
    error_RK2 = abs(yRK2[N]-valor_real)
    
    errores.append([N,error_RK4, error_RK2])
    

print(tabulate(errores, headers=["N", "RK4", "RK2"]))

  N          RK4          RK2
---  -----------  -----------
 10  2.08432e-06  0.00420098
 50  3.56449e-09  0.000178516
100  2.24641e-10  4.49659e-05
500  3.61045e-13  1.80947e-06


Podemos observar que obtenemos valores muy aproximados a la solución exacta con el método de Runge-Kutta de 2 evaluaciones, pero no tanto como el método de Runge-Kutta de 4 evaluaciones.

Además, nótese que para esos valores del arreglo de Butcher, basándonos en lo visto en teoría, el método de Runge-Kutta de 2 evaluaciones coincide con el método de Euler Mejorado (o del Punto Medio).

## Métodos Multipaso Lineales (MML)

### Métodos de tipo Adams

#### 1. A partir de la implementación del algoritmo del método de Adams-Bashforth anterior, realice las modificaciones oportunas para intentar obtener también las implementaciones correspondientes a otros muchos métodos de tipo Adams, como Adams-Moulton (AM), Milne-Simpson, Nyström y/o Newton-Cotes.

In [28]:
# Método de Adams-Bashfort (con k = 2)

def AB2(F,x0,y0,xfinal,N):
    ''' método de Adams-Bashfort (con k = 2) para resolver el PVI
    X,Y     = ecuacionAB(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    def ecuacionAB2(F,x0,y0,y1,h):
        return (3*F(x0+h,y1)-F(x0,y0))/2    
    
    X = np.linspace(x0,xfinal,N+1)
    h = (xfinal-x0)/N
    y1 = y0 + h*F(x0,y0)
    Y = [y0,y1]
    
    for n in range(N-1):
        Y.append(Y[n+1] + h*ecuacionAB2(F,X[n],Y[n],Y[n+1],h))
        
    return np.array(X),np.array(Y)

In [29]:
# Método de Adams-Moulton (con k = 1)

def AM(F,x0,y0,xfinal,N):    
    ''' método de Adams-Moulton (con k = 1) para resolver el PVI
    X,Y     = ecuacionAM(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    def ecuacionAM(F,x0,y0,t,h):
        return 1/2*(F(x0,y0)+F(x0+h,t))
    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    t = sp.Symbol('t')
    
    for n in range(N): 
        Y.append(sp.solve(Y[n]+h*ecuacionAM(F,X[n],Y[n],t,h)-t,t)[0])
    
    return np.array(X),np.array(Y)

In [30]:
# Método de Milne-Simpson

def MS(F,x0,y0,xfinal,N):    
    ''' método de Milne-Simpson para resolver el PVI
    X,Y     = ecuacionAM(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    def ecuacionMS(F,x0,y0,y1,y2,h):
        return (F(x0,y0)+4*F(x0+h,y1)+F(x0+2*h,y2))/3
    
    X = np.linspace(x0,xfinal,N+1)
    h = (xfinal-x0)/N
    y1 = y0 + h*F(x0,y0)
    y2 = y1 + h*F(x0+h,y1)
    Y = [y0,y1]
    
    t = sp.Symbol('t')
    
    for n in range(N):
        Y.append(sp.solve(Y[n]+h*ecuacionMS(F,X[n],Y[n],Y[n+1],t,h)-t,t)[0])
    
    return np.array(X),np.array(Y)

In [31]:
# Método de Nyström

def Nys(F,x0,y0,xfinal,N):    
    ''' método de Nyström para resolver el PVI
    X,Y     = ecuacionAM(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    def ecuacionNys(F,x0,y1,h):
        return 2*F(x0+h,y1)
    
    X = np.linspace(x0,xfinal,N+1)
    h = (xfinal-x0)/N
    y1 = y0 + h*F(x0,y0)
    Y = [y0,y1]
    
    t = sp.Symbol('t')
    
    for n in range(N):  
        Y.append(sp.solve(Y[n]+h*ecuacionNys(F,X[n],Y[n+1],h)-t,t)[0])
    
    return np.array(X),np.array(Y)

In [32]:
# Método de Newton-Cotes

def NC(F,x0,y0,xfinal,N):    
    ''' método de Newton-Cotes para resolver el PVI
    X,Y     = ecuacionAM(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    def ecuacionNC(F,x0,y0,y1,h):
        return (F(x0,y0) + F(x0+h,y1))/2
    
    X = np.linspace(x0,xfinal,N+1)
    Y = [y0]
    h = (xfinal-x0)/N
    
    t = sp.Symbol('t')
    
    for n in range(N):  
        Y.append(sp.solve(Y[n]+h*ecuacionNC(F,X[n],Y[n],t,h)-t,t)[0])
    
    return np.array(X),np.array(Y)

In [33]:
ENES = [10,50,100,500]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yAB2 = AB2(f,a,ya, b, N)
    xx, yAM = AM(f,a,ya, b, N)
    xx, yMS = MS(f,a,ya, b, N)
    xx, yNys = Nys(f,a,ya, b, N)
    xx, yNC = NC(f,a,ya, b, N)
    
    error_AB2 = abs(yAB2[N]-valor_real)
    error_AM = abs(yAM[N]-valor_real)
    error_MS = abs(yMS[N]-valor_real)
    error_Nys = abs(yNys[N]-valor_real)
    error_NC = abs(yNC[N]-valor_real)

    
    errores.append([N, error_AB2, error_AM, error_MS, error_Nys, error_NC])
    

print(tabulate(errores, headers=["N","Adams-Bashfort (k=2)", "Adams-Moulton (k=1)", "Milne-Simpson", "Nyström", "Newton-Cotes"]))

  N    Adams-Bashfort (k=2)    Adams-Moulton (k=1)    Milne-Simpson      Nyström    Newton-Cotes
---  ----------------------  ---------------------  ---------------  -----------  --------------
 10             0.0226833              0.00226959       0.00499273   0.0103261       0.00226959
 50             0.000980007            9.06163e-05      0.000200156  0.000416129     9.06163e-05
100             0.000247112            2.26528e-05      5.00426e-05  0.000104057     2.26528e-05
500             9.95068e-06            9.06095e-07      2.00175e-06  4.16258e-06     9.06095e-07


A la hora de ejecutar estos métodos podemos ver que el tiempo de cómputo aumenta mucho con respecto a otros (especialmentes aquellos que son implícitos).

Por otro lado, se puede observar que el error cometido es bastante similar en todos los métodos para los diferentes valores de N.

## Métodos Predictor-Corrector

#### 1. Intente la implementación del algoritmo de tipo Predictor-Corrector anterior, o cualquier otro adecuado, eligiendo convenientemente los otros métodos necesarios para proporcionar los valores previos necesarios.

Implementaremos a continuación un método MML Predictor-Corrector particular, combinando un predictor AB de 5 pasos con un corrector AM de 4, y aplicando una sólo corrección en cada iteración:

P: $y_{n+5}^{(0)} = y_{n+4} + \frac{h}{720} (1901 f_{n+4} -2774 f_{n+3} +2616 f_{n+2} - 1274 f_{n+1} + 251 f_n)$

C$^1$: $y_{n+5} = y_{n+4} + \frac{h}{720} (251 f(t_{n+5},y_{n+5}^{(0)}) + 646 f_{n+4} -264 f_{n+3} +106 f_{n+2} - 19 f_{n+1}$ 

In [34]:
def predictor_corrector(F,x0,y0,xfinal,N):    
    ''' método predictor-corrector para resolver el PVI
    X,Y     = ecuacionAM(F,x0,y0,xfinal,N).
    {y}'    = {F(x,{y})}, donde
    {y}     = {y[0],y[1],...y[N-1]}.
    x0,y0   = condiciones iniciales 
    xfinal  = valor final de la variable x
    h       = incremento de x usado para la integración
    F       = función suplida por el usuario que devuelve 
            el array F(x,y) = {y'[0],y'[1],...,y'[N-1]}.
    '''
    
    # Predictor
    def P(F,x0,y0,y1,y2,y3,y4,h):
        return (1901*F(x0+4*h,y4)-2774*F(x0+3*h,y3)+2616*F(x0+2*h,y2)-1274*F(x0+h,y1)+251*F(x0,y0))/720
    
    # Corrector
    def C(F,x0,y0,y1,y2,y3,y4,y5,h):
        return (251*F(x0+5*h,y5)+646*F(x0+4*h,y4)-264*F(x0+3*h,y3)+106*F(x0+2*h,y2)-19*F(x0+h,y1))/720
    
    X = np.linspace(x0,xfinal,N+1)
    h = (xfinal-x0)/N
    y1 = y0 + h*F(x0,y0)
    y2 = y1 + h*F(x0+h,y1)
    y3 = y2 + h*F(x0+2*h,y2)
    y4 = y3 + h*F(x0+3*h,y3)
    Y = [y0,y1,y2,y3,y4]
    
    t = sp.Symbol('t')
    
    for n in range(N-4):  
        y5 = Y[n+4]+h*P(F,X[n],Y[n],Y[n+1],Y[n+2],Y[n+3],Y[n+4],h)
        Y.append(Y[n+4]+h*C(F,X[n],Y[n],Y[n+1],Y[n+2],Y[n+3],Y[n+4],y5,h))
    
    return np.array(X),np.array(Y)

In [35]:
ENES = [10,50,100,500]
errores = []
valor_real = np.exp(1)

for N in ENES:
    xx, yPredictorCorrector = predictor_corrector(f,a,ya, b, N)
    
    error_PredictorCorrector = abs(yPredictorCorrector[N]-valor_real)

    errores.append([N, error_PredictorCorrector])
    

print(tabulate(errores, headers=["N","Predictor-Corrector"]))

  N    Predictor-Corrector
---  ---------------------
 10            0.0506218
 50            0.0021461
100            0.000540118
500            2.17181e-05


Podemos observar que el error cometido es mayor que algunos otros métodos que hayamos visto. Sin embargo, al ser un método de 5 pasos tenemos una rápidez en los cálculos mucho mayor que en otros métodos de un paso (esto se nota al ejecutarlo).